In [ ]:
import torch
from diffusers import StableDiffusionXLPipeline
from scheduling_tcd import TCDScheduler 

In [ ]:
device = "cuda"
base_model_id = "stabilityai/stable-diffusion-xl-base-1.0"
tcd_lora_id = "h1t/TCD-SDXL-LoRA"

In [ ]:
# Load the diffusers pipeline for text-to-image generation.
pipe = StableDiffusionXLPipeline.from_pretrained(base_model_id, torch_dtype=torch.float16, variant="fp16").to(device)

In [ ]:
# Set the scheduler to TCDScheduler.
pipe.scheduler = TCDScheduler.from_config(pipe.scheduler.config)

# Load the TCD-LoRA weights for the model.
pipe.load_lora_weights(tcd_lora_id)
pipe.fuse_lora()

In [ ]:
prompt = "Beautiful woman, bubblegum pink, lemon yellow, minty blue, futuristic, high-detail, epic composition, watercolor."

# Perform inference with the pipeline.
image = pipe(
    prompt=prompt,
    num_inference_steps=4,
    guidance_scale=0,
    # Eta (referred to as `gamma` in the paper) is used to control the stochasticity in every step.
    # A value of 0.3 often yields good results.
    # We recommend using a higher eta when increasing the number of inference steps.
    eta=0.3, 
    generator=torch.Generator(device=device).manual_seed(0),
).images[0]